In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from time import sleep
import pandas as pd
import re

爬取二手房数据

In [34]:
service = Service(r'D:\anaconda3\chromedriver.exe') 
driver = webdriver.Chrome(service=service)
driver.maximize_window()
driver.get('https://sh.esf.fang.com/')

In [35]:

#subway = driver.find_element(By.XPATH,'//*[@id="r011"]')
#actions = ActionChains(driver)
#actions.move_to_element(subway).perform()
link_1 = driver.find_element(By.LINK_TEXT,'浦东')
link_1.click()
sleep(0.3)
link_2 = driver.find_element(By.LINK_TEXT,'世博')
link_2.click()
sleep(0.3)


In [36]:
page = driver.find_element(By.XPATH,'/html/body/div[4]/div[4]/div[5]/div/span[10]')
page_number = min(int(re.search(r"共(\d+)页", page.text).group(1)),20)

all_page_information = []
count = 0

while count < page_number:

    # 摘取所有信息
    table = driver.find_element(By.CLASS_NAME,'shop_list.shop_list_4')
    rows = table.find_elements(By.XPATH,'./ *[@class="clearfix"]') # Direct child element
    House_discription = []
    House_information = []
    House_location = []
    House_characteristic = []
    House_price = []
    for row in rows:
        discription = row.find_element(By.CLASS_NAME,'tit_shop')
        House_discription.append(discription.text)
        information = row.find_element(By.CLASS_NAME,'tel_shop')
        House_information.append(information.text)
        location = row.find_element(By.CLASS_NAME,'add_shop')
        House_location.append(location.text)
        characteristic = row.find_element(By.CLASS_NAME,'clearfix.label')
        House_characteristic.append(characteristic.text)
        price = row.find_element(By.CLASS_NAME,'price_right')
        House_price.append(price.text)
    df_discription = pd.DataFrame(House_discription,columns=['描述'])
    df_information = pd.DataFrame(House_information,columns=['House_information'])
    df_location = pd.DataFrame(House_location,columns=['House_location'])
    df_characteristic = pd.DataFrame(House_characteristic,columns=['House_characteristc'])
    df_price = pd.DataFrame(House_price,columns=['House_price'])
    
    # 字段处理
    information_split = []
    for s in df_information['House_information']:
        # 初始化所有字段为None（默认不符合格式）
        bed_room_number = None
        living_room_number = None
        room_match2 = None
        layer = None
        all_layer = None
        direction_match = None
        time_match = None
        name = None
    
        try:
            split_list = s.split('|')
            
            room_hall = split_list[0].strip() 
            room_match1 = re.match(r"(\d+)室(\d+)厅", room_hall) 
            bed_room_number = int(room_match1.group(1))
            living_room_number = int(room_match1.group(2))
            
            room_area = split_list[1].strip()
            area_match = re.search(r"(\d+\.\d+|\d+)", room_area)
            room_match2 = float(area_match.group(1))
            
            floor_str = split_list[2].strip()
            layer_match = re.search(r"(.+?)\（共", floor_str) 
            all_layer_match = re.search(r"共(\d+)层", floor_str)
            layer = layer_match.group(1).strip()
            all_layer = int(all_layer_match.group(1))
            
            direction = split_list[3].strip()
            direction_match = direction 
            
            time = split_list[4].strip()
            time_match = re.search(r'\d+', time)
            time_match = int(time_match.group())
            
            name = split_list[5].strip()
        
        except Exception as e:
            pass

        info_group = [bed_room_number,living_room_number,room_match2,layer,all_layer,direction_match,time_match,name]
        information_split.append(info_group)
    df_information_split = pd.DataFrame(information_split)
    new_columns = ['室数','厅数','面积','层级','总层数','朝向','始建年份','出售人']
    df_information_split.columns = new_columns

    location_split = []
    for s in df_location['House_location']:
        try:
            split_list = re.split(r'[\n ]', s,maxsplit=2)
            while len(split_list) < 3:
                split_list[len(split_list)-1:len(split_list)-1]=[None]
            characteristic_split.append(split_list)
            location_split.append(split_list)
        except:
            location_split.append([None,None,None])
    df_location_split = pd.DataFrame(location_split)
    new_columns = ['小区名称','路段','位置']
    df_location_split.columns = new_columns

    characteristic_split = []
    for s in df_characteristic['House_characteristc']:
        try:
            split_list = s.split('\n')
            last_part = split_list[-1]
            distance = int(re.search(r'约(\d+)', last_part).group(1))
            split_list[-1] = distance
            while len(split_list) < 6:
                split_list[len(split_list)-1:len(split_list)-1]=[None]
            characteristic_split.append(split_list)
        except:
            characteristic_split.append([None,None,None,None,None,None])
    df_characteristic_split = pd.DataFrame(characteristic_split)
    new_columns = ['房产证登记年份','特征1','特征2','特征3','特征4','距离地铁站距离']
    df_characteristic_split.columns = new_columns

    price_split = []
    for s in df_price['House_price']:
        split_list = s.split('\n')
        price_total = int(re.search(r'\d+',split_list[0]).group())*10000
        price_square = int(re.search(r'\d+',split_list[1]).group())
        split_list = [price_total,price_square]
        price_split.append(split_list)
    df_price_split = pd.DataFrame(price_split)
    new_columns = ['总价格','价格每平方米']
    df_price_split.columns = new_columns

    #整合本页信息
    one_page_information = pd.concat([df_discription,df_information_split,df_location_split,df_characteristic_split,df_price_split],axis=1)

    #加入所有信息的list
    all_page_information.append(one_page_information)

    #翻页
    turn_page = driver.find_element(By.LINK_TEXT,"下一页")
    turn_page.click()
    sleep(2)
    count = count + 1

df_all_page_information = pd.concat(all_page_information,axis=0)
df_all_page_information.to_excel("second_hand_house.xlsx",index=False)


    


    


    

    

    

    

In [21]:
df_location_split

,0,1,2,3
0,德州四村,世博,成山路24弄,None
1,上南五村,世博,上南五村,None
2,上钢五村,世博,长清路昌里路,None
3,新世纪花苑三期,世博,耀华路579弄,None
4,新世纪花苑三期,世博,耀华路579弄,None
5,上南九村,世博,上南九村,None
6,博爱家园,世博,东明路355弄1-70号,None
7,凯德嘉博名邸,世博,齐恒路177弄,None
8,恒大小区,世博,恒大小区,None
9,建业新村,世博,杨高南路2030弄、杨高南路2140弄,None
